In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import itertools as it
import sympy as sp 
import pandas as pd
from sympy import symbols, Function, diff, conjugate, FiniteSet, simplify, Poly, series, re
from sympy import *
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import nC
from sympy.combinatorics.named_groups import CyclicGroup
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt

$n$ = dimension ($\mathbb{R}^n$ or $\mathbb{C}^n$) \
$k$ = real degree \
$(p,q)$ = complex bidegree

In [2]:
####### Change constants here #######
n = 4
k = 3
p = 2
q = 1
#####################################

# Setting up symbols
f = symbols('f', cls = Function)
x,y,z,w,zeta = symbols('x y z w zeta')
zs = symbols(f'z1:{n+1}', complex=True)
zbar = symbols(f'zbar1:{n+1}', complex=True)
ls = symbols(f'l1:{n+1}', int=True)

# Newton potential (R^n) 
mag_real = 0
for i in range(n):
    mag_real += zs[i]**2 ## Euclidean norm squared
np_real = sp.sqrt(mag_real)**(2-n)

# Newton potential (C^n)
mag_complex = 0
for i in range(n):
    mag_complex += zs[i]*zbar[i] ## Complex modulus squared
np_complex = sp.sqrt(mag_complex)**(2-2*n)

In [3]:
# Cyclic group (roots of unity)
m=5
zeta = exp(2*pi*I/m)
g = diag(zeta, zeta**(-1))
G = FiniteSet()
for i in range(m):
    G += FiniteSet(g**i)
G

{Matrix([
[1, 0],
[0, 1]]), Matrix([
[exp(-4*I*pi/5),             0],
[             0, exp(4*I*pi/5)]]), Matrix([
[exp(-2*I*pi/5),             0],
[             0, exp(2*I*pi/5)]]), Matrix([
[exp(2*I*pi/5),              0],
[            0, exp(-2*I*pi/5)]]), Matrix([
[exp(4*I*pi/5),              0],
[            0, exp(-4*I*pi/5)]])}

In [4]:
# Multi index differentiation
def D(f,alpha,zs):
    df = f
    for i in range(n):
        df = diff(df,(zs[i],alpha[i]))
    return df

In [5]:
# Returns valid n-tuples
# https://www.geeksforgeeks.org/python-program-to-convert-dictionary-to-list-by-repeating-keys-corresponding-value-times/

def get_alpha_beta(deg):
    alphas = FiniteSet()
    for part in partitions(deg, m=n):
        part = list(it.chain.from_iterable(it.repeat(k, v) for k, v in part.items()))
        while len(part) < n:
            part.append(0)
        for perm in it.permutations(part):
            alphas += FiniteSet(perm)
    return alphas

In [6]:
def make_polynomial(f, mag):
    f = sp.numer(f).subs(mag, x)
    while f.is_polynomial(zs,zbar,x) == False:
        f = simplify(f * x)
        f = sp.numer(f).subs(mag, x)
    return expand(f.subs(x,abs(z)**2))

In [7]:
# For H_k(R^n)
def real_dim():
    return nC(n+k-1,k) - nC(n+k-3,k-2)

In [8]:
def dim_H_k():
    return nC(2*n+k-1,k) - nC(2*n+k-3,k-2)

In [9]:
def dim_H_pq():
    return nC(n+p-1,p)*nC(n+q-1,q) - nC(n+p-2,p-1)*nC(n+q-2,q-1)

In [10]:
# Finding basis for H_k(R^n) - see Axler's Harmonic Function Theory p. 92
def real_basis():
    basis = FiniteSet()
    alphas = get_alpha_beta(k)
    for a in alphas:
        if a[0] > 1:
            alphas -= FiniteSet(a)
    print(f'# of elements = {len(alphas)}')
    print(f'Dimension from combinatorial expression = {real_dim()} \n')
    for a in alphas:
        d = D(np_real, a, zs)
        d = make_polynomial(d, mag_real)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [11]:
# Finding basis for H_p,q(C^n)
def basis_H_pq():
    basis = FiniteSet()
    alphas = get_alpha_beta(p)
    betas = get_alpha_beta(q)
    ab = FiniteSet()
    for a in alphas:
        for b in betas:
            if a[0]==0 or b[0]==0:
                ab += FiniteSet((a,b))
    print(f'# of elements = {len(ab)}')
    print(f'Dimension from combinatorial expression = {dim_H_pq()} \n')
    for (a,b) in ab:
        d = D(np_complex,b,zs)
        d = D(d,a,zbar)
        d = make_polynomial(d, mag_complex)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [12]:
def box(f):
    laplacian = 0
    for i in range(n):
        laplacian += diff(diff(f,zbar[i]),zs[i])
    return laplacian

In [13]:
# Closed form from 2018 REU paper
def box_b(f):
    box_b = 0
    # first term
    for i in range(n): 
        box_b += -2*diff(diff(f,zbar[i]), zs[i])
        
    # second term
    for i in range(n): 
        box_b += 2*diff(zs[i]*q*f, zs[i])
        
    # third term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += 2*diff(zs[a]*zbar[i]*diff(f,zbar[i]), zs[a])
            
    # fourth term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += -2*diff(zs[a]*zbar[i]*zs[i]*q*f, zs[a])
    return box_b

In [14]:
# Just in case :)
def is_harmonic(f):
    return (box_b(f)/f).simplify().is_polynomial(mag_complex)

In [15]:
B = basis_H_pq()
B

# of elements = 36
Dimension from combinatorial expression = 36 

-60*z4**2*zbar4 + 24*z4*Abs(z)**2 

-60*z4**2*zbar3 

-60*z4**2*zbar2 

-60*z4**2*zbar1 

-60*z3*z4*zbar4 + 12*z3*Abs(z)**2 

-60*z3*z4*zbar3 + 12*z4*Abs(z)**2 

-60*z3*z4*zbar2 

-60*z3*z4*zbar1 

-60*z3**2*zbar4 

-60*z3**2*zbar3 + 24*z3*Abs(z)**2 

-60*z3**2*zbar2 

-60*z3**2*zbar1 

-60*z2*z4*zbar4 + 12*z2*Abs(z)**2 

-60*z2*z4*zbar3 

-60*z2*z4*zbar2 + 12*z4*Abs(z)**2 

-60*z2*z4*zbar1 

-60*z2*z3*zbar4 

-60*z2*z3*zbar3 + 12*z2*Abs(z)**2 

-60*z2*z3*zbar2 + 12*z3*Abs(z)**2 

-60*z2*z3*zbar1 

-60*z2**2*zbar4 

-60*z2**2*zbar3 

-60*z2**2*zbar2 + 24*z2*Abs(z)**2 

-60*z2**2*zbar1 

-60*z1*z4*zbar4 + 12*z1*Abs(z)**2 

-60*z1*z4*zbar3 

-60*z1*z4*zbar2 

-60*z1*z3*zbar4 

-60*z1*z3*zbar3 + 12*z1*Abs(z)**2 

-60*z1*z3*zbar2 

-60*z1*z2*zbar4 

-60*z1*z2*zbar3 

-60*z1*z2*zbar2 + 12*z1*Abs(z)**2 

-60*z1**2*zbar4 

-60*z1**2*zbar3 

-60*z1**2*zbar2 



{-60*z1**2*zbar2, -60*z1**2*zbar3, -60*z1**2*zbar4, -60*z2**2*zbar1, -60*z2**2*zbar3, -60*z2**2*zbar4, -60*z3**2*zbar1, -60*z3**2*zbar2, -60*z3**2*zbar4, -60*z4**2*zbar1, -60*z4**2*zbar2, -60*z4**2*zbar3, -60*z1*z2*zbar3, -60*z1*z2*zbar4, -60*z1*z3*zbar2, -60*z1*z3*zbar4, -60*z1*z4*zbar2, -60*z1*z4*zbar3, -60*z2*z3*zbar1, -60*z2*z3*zbar4, -60*z2*z4*zbar1, -60*z2*z4*zbar3, -60*z3*z4*zbar1, -60*z3*z4*zbar2, -60*z2**2*zbar2 + 24*z2*Abs(z)**2, -60*z3**2*zbar3 + 24*z3*Abs(z)**2, -60*z4**2*zbar4 + 24*z4*Abs(z)**2, -60*z1*z2*zbar2 + 12*z1*Abs(z)**2, -60*z1*z3*zbar3 + 12*z1*Abs(z)**2, -60*z1*z4*zbar4 + 12*z1*Abs(z)**2, -60*z2*z3*zbar2 + 12*z3*Abs(z)**2, -60*z2*z3*zbar3 + 12*z2*Abs(z)**2, -60*z2*z4*zbar2 + 12*z4*Abs(z)**2, -60*z2*z4*zbar4 + 12*z2*Abs(z)**2, -60*z3*z4*zbar3 + 12*z4*Abs(z)**2, -60*z3*z4*zbar4 + 12*z3*Abs(z)**2}

In [16]:
def F(k,ls):
    F = 0
    zeta = exp(2*pi*I/k)
    for m in range(k):
        frac = 1-z*w
        for i in range(len(ls)):
            frac /= (z-zeta**(-m*ls[i]))*(w-zeta**(m*ls[i]))
        F += frac
    F = (1/k)*F
    return F

In [17]:
def taylor_polynomial(f,deg):
    table = []
    for i in range(deg+1):
        row = []
        for j in range(deg+1-i):
            d = diff(f,w,j)
            d = diff(d,z,i)
            d = lambdify([z,w],d)
            term = re(d(0,0)/(factorial(i)*factorial(j))).round()
            row.append(term)
        table.append(row)
    return table

In [18]:
f = F(5,[1,-1])
f

0.4*(-w*z + 1)/((w - exp(-2*I*pi/5))*(w - exp(2*I*pi/5))*(z - exp(-2*I*pi/5))*(z - exp(2*I*pi/5))) + 0.4*(-w*z + 1)/((w - exp(-4*I*pi/5))*(w - exp(4*I*pi/5))*(z - exp(-4*I*pi/5))*(z - exp(4*I*pi/5))) + 0.2*(-w*z + 1)/((w - 1)**2*(z - 1)**2)

In [19]:
T=taylor_polynomial(f,3)
pd.DataFrame(T)

,0,1,2,3
0,1,0,1,0
1,0,1,0,None
2,1,0,None,None
3,0,None,None,None
